In [34]:
#REFS:
#Wildon, Vertices of Specht Modules and Blocks of the Symmetric Group
#Murphy, The Idempotents of the Symmetric Group and Nakayama's Conjecture
#Murphy, A new construction of Young's seminormal representation of the symmetric groups

In [188]:
SGA_Q3 = SymmetricGroupAlgebra(QQ,3)

In [147]:
SGA_Q3.dft()

[   1    1    1    1    1    1]
[   1  1/2   -1 -1/2 -1/2  1/2]
[   0  3/4    0  3/4 -3/4 -3/4]
[   0    1    0   -1    1   -1]
[   1 -1/2    1 -1/2 -1/2 -1/2]
[   1   -1   -1    1    1   -1]

In [148]:
SGA_F7 = SymmetricGroupAlgebra(GF(7),3)

In [149]:
SGA_F7.dft()

[1 1 1 1 1 1]
[1 4 6 3 3 4]
[0 6 0 6 1 1]
[0 1 0 6 1 6]
[1 3 1 3 3 3]
[1 6 6 1 1 6]

In [150]:
SGA_F7.specht_module(Partition([1,1,1]))

Specht module of [(0, 0), (1, 0), (2, 0)] over Finite Field of size 7

In [151]:
SGA_F3 = SymmetricGroupAlgebra(GF(3),3)

In [152]:
#one cannot perform the DFT when p | n!
try:
    SGA_F3.dft()
except ZeroDivisionError:
    print("Modular case not handled!")

Modular case not handled!


In [43]:
#Theorem 3.3, "Nakayama's Conjecture", pg. 8, Wildon, Vertices of Specht Modules and Blocks of the Symmetric Group:

#Let p be a prime. The p-blocks of the symmetric group Sn are labelled by pairs (\gamma,w), 
#where \gamma is a p-core and w \in \mathbb{N}_0 is the associated weight, such that |\gamma| + wp = n. 
#Thus S^\lambda lies in the block labelled by (\gamma,w) if and only if \lambda has p-core \gamma and weight w.
#
#---
#
#Theorem 2.1, Murphy, The Idempotents of the Symmetric Group and Nakayama's Conjecture
#
#F_j^i = \sum_{\mu,k | t_k^\mu \in T_j^i} E_k^\mu 
#H^i = \sum_j F_j^i 
#
#{F_j^i} is a complete set of orthogonal idempotents of RS_n, similarly for {\overline{F}_j^i}.
#
#where
#
#E_i^\mu = \prod_{c = -n+1}^{n-1} \prod_{u | a_{ui}^\mu \ne c} \frac{c-L_u}{c-\alpha_{ui}^\mu}
#
#L_u = (1,u) + (2,u) + ... + (u-1,u), a sum of transpositions
#
#T_j^i are tableaux belonging to equivalence classes B_i, where B_i are equivalences classes of
#partitions given by \tilde_p
#
#\labmda \tidle_p \mu iff the classes (j-i) of each node have the same residue mod p
#
#The class of the node occupied by u in t_i^\mu is \alpha_{ui}^\mu where t_1^\mu, ..., t_d^\mu are the standard \mu-tableaux 
#in the ordering in pg. 288 of "A new construction of Young's seminormal representation of the symmetric groups".
#
#The "class" of a node (i,j) in a Young diagram \mu is the difference j-i.
#
#d is the number of "standard \mu-tableaux", given by the Hook-length formula.
#
#---
#
#Theorem 2.8 (Nakayama's Conjecture): {\overline{H}^i} is a complete set of primitive orthogonal central idempotents of \overline{K}S_n,
#and S_{\overline{K}}^\mu, S_{\overline{K}}^\lambda belong to the same block of \overline{K}S_n if and only if \mu \tilde_p \lambda.
#
#---

In [44]:
#define the element \alpha_{ui}^\mu
#which is the class of u in the i^th standard Tableaux corresponding to mu
#the class of a node is j-i where i is the row and j is the column
def alpha(mu,u,i):
    tab = StandardTableaux(mu)[i]
    for row in range(len(tab)):
        for col in range(len(tab[row])):
            if u == tab[row][col]:
                return col - row

In [193]:
#define the elment L_u = (1,u) + (2,u) + ... + (u-1,u) in K[S_n]
def L(u,n):
    SGA = SymmetricGroupAlgebra(QQ,n)
    return sum(SGA(PermutationGroupElement(f"({i},{u})")) for i in range(1,u))

In [196]:
#from Murphy
def E(mu,i,n):
    print(n)
    return prod(prod((c-L(u,n))/(c-alpha(mu,u,i)) for u in range(1,n+1) if alpha(mu,u,i) != c) for c in range(-n+1,n))

In [197]:
#check idempotency
mu = Partition([4,1])
E(mu,0,3)*E(mu,0,3) == E(mu,0,3)

3
3
3


True

In [48]:
#helper function to determine if diagram comes from a partition
def diagram_is_from_partition(diag):
    #handle empty diagram
    if len(diag) == 0:
        return True
    from_partition = True
    #check if rows start at 0 and have any gaps
    row_labels = {node[0] for node in diag}
    from_partition &= min(row_labels) == 0
    from_partition &= len(row_labels) == max(row_labels) - min(row_labels) + 1
    #for each row, check if columns have any gaps, and start at 0
    part = []
    for row in row_labels:
        col_labels = {node[1] for node in diag if node[0]==row}
        part.append(len(col_labels))
        from_partition &= min(col_labels) == 0
        from_partition &= len(col_labels) == max(col_labels) - min(col_labels) + 1
    from_partition &= all(part[i] >= part[i+1] for i in range(len(part) - 1))
    return from_partition

In [49]:
#given a diagram which comes from a partition, give the partition
def to_partition(diag):
    assert diagram_is_from_partition(diag)
    return Partition([max({node[1] for node in diag if node[0]==row})+1 for row in {node[0] for node in diag}])

In [65]:
#find the p-core of a partition
#a "rim p-hook" is:
#connected set of p nodes
#in the rim (the node (i+1,j+1) is not in the diagram)
#whose removal leaves a valid young diagram
from sage.combinat.diagram import Diagram, NorthwestDiagrams
def core(mu,p):
    #get the diagram associated to the partition
    diag = NorthwestDiagrams().from_partition(mu)
    #find the rim of the diagram/partition
    rim = []
    for node in diag:
        if (node[0]+1,node[1]+1) not in diag:
            rim.append(node)
    #find rim p-hooks
    p_hooks = []
    for start in rim:
        p_hook = [start]
        for step in range(p-1):
            current = p_hook[step]
            down = (current[0]+1,current[1])
            left = (current[0],current[1]-1)
            if (not down in rim) and (not left in rim):
                break
            if down in rim:
                p_hook.append(down)
                continue
            if left in rim:
                p_hook.append(left)
                continue
        if len(p_hook) == p:
            p_hooks.append(p_hook)
    #determine if removal of candidate p_hook results in a diagram
    valid_p_hook_removed_diags = []
    for p_hook in p_hooks:
        diag_minus_p_hook = Diagram([node for node in diag if node not in p_hook])
        if diagram_is_from_partition(diag_minus_p_hook):
            valid_p_hook_removed_diags.append(diag_minus_p_hook)
    #recursive step: if no p-hooks, return original diagram
    if len(valid_p_hook_removed_diags) == 0:
        return to_partition(diag)
    #recursive step: if diagram is unchanged, return, otherwise keep going
    for valid_p_hook_removed_diag in valid_p_hook_removed_diags:
        partition = to_partition(valid_p_hook_removed_diag)
        return core(partition,p)

In [51]:
diag = NorthwestDiagrams().from_partition(Partition([6,6,5,1,1]))

In [52]:
(4,1) in diag

False

In [67]:
core(Partition([4,1]),3)

[1, 1]

In [54]:
#determine when two partitions are p-equivalent, i.e. have the same p-core
#look at canonical tableaus (place 1,...,n in top-to-bottom, left-to-right)
#compute p-residues of classes j-i \mod p
#using the method described in Littlewood '51
#just check if the sets have the same residue classes
#BUG: for n=5, p=3, the partition [1,1,1,1,1] is in two classe with residue {0,1,2}
from multiset import *
def p_equiv(mu_1, mu_2,p,method="residue"):
    if method=="core":
        return core(mu_1,p) == core(mu_2,p)
    if method=="residue":
        max_len = max(len(mu_1),len(mu_2))
        #zero pad partitions
        mu_1 += [0]*(max_len - len(mu_1))
        mu_2 += [0]*(max_len - len(mu_2))
        #compute residue sets
        residues_1 = set([(mu_1[i]+max_len-1-i) % p for i in range(max_len)])
        residues_2 = set([(mu_2[i]+max_len-1-i) % p for i in range(max_len)])
        return residues_1 == residues_2
    if method=="multiset":
        diag_1=NorthwestDiagrams().from_partition(mu_1)
        diag_2=NorthwestDiagrams().from_partition(mu_2)
        res_1 = Multiset([(node[1]-node[0]) % 5 for node in diag_1])
        res_2 = Multiset([(node[1]-node[0]) % 5 for node in diag_2])
        k=0
        while res_1 >= Multiset({i:k for i in range(5)}):
            k += 1
        reduced_1 = res_1 - Multiset({i:k-1 for i in range(5)})
        k=0
        while res_2 >= Multiset({i:k for i in range(5)}):
            k += 1
        reduced_2 = res_2 - Multiset({i:k-1 for i in range(5)})
        return reduced_1 == reduced_2

In [55]:
#equivalence classes of all partitions under \tilde_p
#two partitions are equivalent if they have the same p-core
def B(n,p):
    equiv_classes = []
    for part_1 in Partitions(n):
        equiv_class_found = False
        for equiv_class in equiv_classes:
            for part_2 in equiv_class:
                if p_equiv(part_1,part_2,p,method="core"):
                    equiv_class.append(part_1)
                    equiv_class_found = True
                    break
        if not equiv_class_found:
            equiv_classes.append([part_1])
    return equiv_classes

In [56]:
B(5,3)

[[[5], [2, 2, 1], [2, 1, 1, 1]],
 [[4, 1], [3, 2], [1, 1, 1, 1, 1]],
 [[3, 1, 1]]]

In [57]:
list(StandardTableaux(mu))

[[[1, 3], [2]], [[1, 2], [3]]]

In [58]:
#in each equivalence class B_i, we have p-equivalent partitions
#each partition in B_i has associated standard tableaux
#we obtain equivalences classes T_j^i by extending ~_p from partitions to tableaux
#define t ~_p t^* if every u <= n occupies a node of the same p-class in both t and t^*
#TO-DO
def equiv_tableaux(t1,t2,p):
    equiv = True
    #ensure talbeaux have same shape
    assert [len(row) for row in t1] == [len(row) for row in t2]
    #find index of u in t1 and t2
    for u in range(1,n+1):
        for i in range(len(t1)):
            for j in range(len(t1[i])):
                if t1[i][j] == u:
                    res_1 = (j-i) % p
                if t2[i][j] == u:
                    res_2 = (j-i) % p
        equiv &= res_1 == res_2
    return equiv

In [59]:
equiv_tableaux(list(StandardTableaux(mu))[0],list(StandardTableaux(mu))[1],5)

False

In [88]:
#flatten a list
def flatten(l):
    return [item for sublist in l for item in sublist]

In [97]:
#equivalence classes of all partitions under \tilde_p
#two partitions are equivalent if they have the same p-core
def T(i,n,p):
    partition_equiv_class = B(n,p)[i]
    equiv_classes_list = []
    for mu in partition_equiv_class:
        std_tableaux = StandardTableaux(mu)
        equiv_classes = []
        for tab_1 in std_tableaux:
            equiv_class_found = False
            for equiv_class in equiv_classes:
                for tab_2 in equiv_class:
                    if equiv_tableaux(tab_1,tab_2,p):
                        equiv_class.append(tab_1)
                        equiv_class_found = True
                        break
            if not equiv_class_found:
                equiv_classes.append([tab_1])
        equiv_classes_list.append(equiv_classes)
    return flatten(equiv_classes_list)

In [61]:
partition_equiv_class = B(5,3)[0]

In [62]:
std_tableaux = [StandardTableaux(mu) for mu in partition_equiv_class]

In [63]:
std_tableaux

[Standard tableaux of shape [5],
 Standard tableaux of shape [2, 2, 1],
 Standard tableaux of shape [2, 1, 1, 1]]

In [103]:
T(0,5,3)[4]

[[[1, 2], [3], [4], [5]]]

In [139]:
#define idempotents F_i^j
def F(i,j,n,p):
    F_ij = 0
    tableaux = T(i,n,p)[j]
    for t in tableaux:
        mu = [len(l) for l in t]
        std_tableaux = StandardTableaux(mu)
        k = list(std_tableaux).index(Tableau(t))
        F_ij += E(mu,k,n)
    return F_ij

In [140]:
F(0,0,5,3)

ValueError: The permutation has length 3 but its maximal element is 4. Some element may be repeated, or an element is missing, but there is something wrong with its length.